<a href="https://colab.research.google.com/github/JosephChenEcho/gemini_stuff/blob/main/Resume.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import google.generativeai as genai
import requests
from bs4 import BeautifulSoup
import re
import urllib.parse



In [ ]:
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)
model = genai.GenerativeModel('gemini-pro')



In [ ]:
def get_html(url):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Check if the request was successful
        return response.text
    except requests.exceptions.HTTPError as http_err:
        print(f'HTTP error occurred: {http_err}')
    except Exception as err:
        print(f'Other error occurred: {err}')

# Example usage
# url = 'https://www.linkedin.com/jobs/view/data-scientist-decisions-at-lyft-3926349865/'
# html_content = get_html(url)

def get_job_description(url):
  html_content = get_html(url)
  soup = BeautifulSoup(html_content, 'html.parser')
  elements = soup.find('div', class_='show-more-less-html__markup show-more-less-html__markup--clamp-after-5 relative overflow-hidden')
  job_description = str(elements)
  job_description=re.sub(r'<div.*>', '', job_description)
  job_description=re.sub(r'</div>', '', job_description)

  return job_description

def get_resume_score(resume, target_position):

  return f"""
  [System]

  Please act as a professional resume scoring system and provide valuable improve hints.

  [User Question]

  Here is my resume:

  {resume}

  Only answer the three following questions with score from 0-10 and provide improve hints:
  1. Does this resume have Relevant Work Experience for the {target_position} role?
  2. Does this resume have Quantifiable Achievements for the {target_position} role?
  3. Does this resume have Keyword Alignment for the {target_position} role?
  """


In [ ]:
target_position = 'Program Manager'
my_resume = """

"""

In [ ]:
# main function

html_content = get_html(f'https://www.linkedin.com/jobs/search/?keywords={urllib.parse.quote(target_position)}&location=United%20States')
linkedin_view_links = re.findall(r'https:\/\/www\.linkedin\.com\/jobs\/view\/[\w-]+', html_content)
# print(linkedin_view_links)

job_description_list = []
job_description_long_text = ''
for link in linkedin_view_links[:10]:
  # job_description_list.append(get_job_description(link))
  job_description_long_text += get_job_description(link) + '\n'


my_resume_prompt = get_resume_score(my_resume, target_position)

response = model.generate_content(my_resume_prompt)
my_resume_score = response.text
print("\n[My Resume Score]")
print(my_resume_score)

prompt_improve_resume = f"""

[System]

Please act as a professional resume writer.


Here is the resume:

{my_resume}

Here is the requirement for {target_position}:

{job_description_long_text}

Here is the improve hints:

{my_resume_score}

[User Question]

return the resume with improvement in Relevant Work Experience, Quantifiable Achievements, Keyword Alignment
"""


response = model.generate_content(prompt_improve_resume)

improved_resume = response.text
# print(improved_resume)

improved_resume_prompt = get_resume_score(improved_resume, target_position)

response = model.generate_content(improved_resume_prompt)
improved_resume_score = response.text
print("\n[Improved Resume Score]")
print(improved_resume_score)